In [1]:
import scipy.io as sio
import os
import matplotlib.pyplot as plt
import numpy as np

from google.colab import files

In [2]:
machine_data_path = r'/content/data_files/'
try:
    os.mkdir(machine_data_path)
except OSError:
      print ("Creation of the directory %s failed" % machine_data_path)
else:
    print ("Successfully created the directory %s " % machine_data_path)

Successfully created the directory /content/data_files/ 


In [3]:
!wget https://eigenvector.com/wp-content/uploads/2019/06/nir_shootout_2002.mat_.zip

--2021-10-06 07:03:33--  https://eigenvector.com/wp-content/uploads/2019/06/nir_shootout_2002.mat_.zip
Resolving eigenvector.com (eigenvector.com)... 69.163.163.60, 2607:f298:6:a034::eaf:812c
Connecting to eigenvector.com (eigenvector.com)|69.163.163.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5414232 (5.2M) [application/zip]
Saving to: ‘nir_shootout_2002.mat_.zip’

nir_shootout_2002.m 100%[===================>]   5.16M  8.63MB/s    in 0.6s    

2021-10-06 07:03:34 (8.63 MB/s) - ‘nir_shootout_2002.mat_.zip’ saved [5414232/5414232]



In [4]:
!unzip nir_shootout_2002.mat_.zip -d data_files

Archive:  nir_shootout_2002.mat_.zip
  inflating: data_files/nir_shootout_2002.mat  
   creating: data_files/__MACOSX/
  inflating: data_files/__MACOSX/._nir_shootout_2002.mat  


In [7]:
# https://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], sio.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, sio.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, sio.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = sio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [8]:
data_content = loadmat('/content/data_files/nir_shootout_2002.mat')

In [14]:
data_content.keys()

dict_keys(['__header__', '__version__', '__globals__', 'calibrate_1', 'calibrate_2', 'test_1', 'test_2', 'validate_1', 'validate_2', 'calibrate_Y', 'test_Y', 'validate_Y'])

In [15]:
calibrate_1 = data_content['calibrate_1']

In [17]:
calibrate_1.keys()

dict_keys(['name', 'type', 'author', 'date', 'moddate', 'data', 'label', 'axisscale', 'title', 'class', 'include', 'description', 'userdata', 'datasetversion', 'history'])

In [25]:
np.shape(calibrate_1['data'])

(155, 650)